<a href="https://colab.research.google.com/github/AsiMrz/AB-Testeing/blob/data-visualization/data-story.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AB Testing on Mobile Game COOKIE CAT

In [3]:
import numpy as np
import math
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from scipy import stats
from IPython.display import display, Latex

Goal of AB testing: Data analysis on the impact on player retention  with an AB-test. 
Variations diffrent Gates in level 30 and level 40.
Sample: 90,189 players
Condition: randomassignmet to to either gate_30 or gate_40.


# Data Exploration

In [11]:
import pandas as pd
import requests
import io

url = 'https://raw.githubusercontent.com/AsiMrz/AB-Testeing/data/cookie_cats.csv' 
# Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content

df = pd.read_csv(url)
# Dataset is now stored in a Pandas Dataframe
df.head()
# Downloading the csv file from your GitHub account

# Reading the downloaded content and turning it into a pandas dataframe

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


Variables:
**userid** a unique number that identifies each player.
**version** whether the player was put in the control group (gate_30 - a gate at level 30) or the test group (gate_40 - a gate at level 40).
**sum_gamerounds** the number of game rounds played by the player during the first week after installation
**retention_1** did the player come back and play 1 day after installing?
**retention_7** did the player come back and play 7 days after installing?

# 01. Checking Errors

In [12]:
df.info()
# Checking missing Values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


There is no missing value in this dataset.

In [16]:
# Checking if users have been sampled multiple times? ref:https://towardsdatascience.com/ab-testing-with-python-e5964dd66143#3.-Visualising-the-results
session_counts = df['userid'].value_counts(ascending=False) 
multi_users = session_counts[session_counts > 1].count()

print(f'There are {multi_users} users that appear multiple times in the dataset')

There are 0 users that appear multiple times in the dataset


In [15]:
df.groupby('version').count()
# the number of players in each group

,userid,sum_gamerounds,retention_1,retention_7
version,,,,
gate_30,44700,44700,44700,44700
gate_40,45489,45489,45489,45489


When a player installed the game, he or she was randomly assigned to either gate_30 or gate_40. The number of players is roughly the same in the control and test group.

# Player Behavior Analysis
To learn about the behavior of the majority of the players, we will check the percentiles.

In [20]:
df['sum_gamerounds'].describe()

count    90189.000000
mean        51.872457
std        195.050858
min          0.000000
25%          5.000000
50%         16.000000
75%         51.000000
max      49854.000000
Name: sum_gamerounds, dtype: float64